<a href="https://colab.research.google.com/github/Pavithran203/Azure-Node.js-Web-App-Deployment/blob/main/Resume_Classification_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resume Classification System

Libraries imported


In [80]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [81]:
import pandas as pd
df = pd.read_csv('/content/resume_data.csv')
df_model = df[
    [
        'career_objective',
        'skills',
        'responsibilities',
        'degree_names',
        '﻿job_position_name'
    ]
].copy()
# Rename for clarity
df_model.rename(columns={
    'career_objective': 'resume_text',
    'job_position_name': 'category'
}, inplace=True)
df_model.head()

,resume_text,skills,responsibilities,degree_names,﻿job_position_name
0,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",Technical Support\nTroubleshooting\nCollaborat...,['B.Tech'],Senior Software Engineer
1,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...",Machine Learning Leadership\nCross-Functional ...,"['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']",Machine Learning (ML) Engineer
2,NaN,"['Software Development', 'Machine Learning', '...","Trade Marketing Executive\nBrand Visibility, S...",['B.Tech'],"Executive/ Senior Executive- Trade Marketing, ..."
3,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...",Apparel Sourcing\nQuality Garment Sourcing\nRe...,['Computer Applications Specialist Certificate...,Business Development Executive
4,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",iOS Lifecycle\nRequirement Analysis\nNative Fr...,['Bachelor of Business Administration'],Senior iOS Engineer


Data validationa and Cleaning

In [82]:
print(df.columns)

Index(['address', 'career_objective', 'skills', 'educational_institution_name',
       'degree_names', 'passing_years', 'educational_results', 'result_types',
       'major_field_of_studies', 'professional_company_names', 'company_urls',
       'start_dates', 'end_dates', 'related_skils_in_job', 'positions',
       'locations', 'responsibilities', 'extra_curricular_activity_types',
       'extra_curricular_organization_names',
       'extra_curricular_organization_links', 'role_positions', 'languages',
       'proficiency_levels', 'certification_providers', 'certification_skills',
       'online_links', 'issue_dates', 'expiry_dates', '﻿job_position_name',
       'educationaL_requirements', 'experiencere_requirement',
       'age_requirement', 'responsibilities.1', 'skills_required',
       'matched_score'],
      dtype='object')


Shape & Column

In [83]:
df_model.shape
df_model.isnull().sum()

,0
resume_text,4804
skills,56
responsibilities,0
degree_names,84
﻿job_position_name,0


In [84]:
df.columns = df.columns.str.replace('\ufeff', '', regex=True).str.strip()
df_model = df[
    [
        'career_objective',
        'skills',
        'responsibilities',
        'degree_names',
        'job_position_name'
    ]
].copy()
df_model.rename(columns={
    'career_objective': 'resume_text',
    'job_position_name': 'category'
}, inplace=True)
df_model.head()

,resume_text,skills,responsibilities,degree_names,category
0,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",Technical Support\nTroubleshooting\nCollaborat...,['B.Tech'],Senior Software Engineer
1,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...",Machine Learning Leadership\nCross-Functional ...,"['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']",Machine Learning (ML) Engineer
2,NaN,"['Software Development', 'Machine Learning', '...","Trade Marketing Executive\nBrand Visibility, S...",['B.Tech'],"Executive/ Senior Executive- Trade Marketing, ..."
3,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...",Apparel Sourcing\nQuality Garment Sourcing\nRe...,['Computer Applications Specialist Certificate...,Business Development Executive
4,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",iOS Lifecycle\nRequirement Analysis\nNative Fr...,['Bachelor of Business Administration'],Senior iOS Engineer


In [85]:
df_model.columns
df_model = df_model.dropna(subset=['category'])
df_model['resume_text'] = df_model['resume_text'].fillna('')
df_model = df_model[df_model['resume_text'].str.strip() != '']
df_model.reset_index(drop=True, inplace=True)

Data cleaning and Preprocessing

In [86]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [87]:
df_model['clean_text'] = df_model['resume_text'].apply(clean_text)
df_model = df_model[df_model['clean_text'].str.strip() != '']
df_model.reset_index(drop=True, inplace=True)
df_model[['resume_text', 'clean_text']].head()

,resume_text,clean_text
0,Big data analytics working and database wareho...,big data analytics working and database wareho...
1,Fresher looking to join as a data analyst and ...,fresher looking to join as a data analyst and ...
2,To obtain a position in a fast-paced business ...,to obtain a position in a fast paced business ...
3,Professional accountant with an outstanding wo...,professional accountant with an outstanding wo...
4,"To secure an IT specialist, desktop support, n...",to secure an it specialist desktop support net...


Selecting targets

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_model['category'])
label_encoder.classes_

array(['AI Engineer', 'Asst. Manager/ Manger (Administrative)',
       'Business Development Executive', 'Civil Engineer',
       'Data Engineer', 'Data Science Engineer',
       'Database Administrator (DBA)', 'DevOps Engineer',
       'Executive - VAT',
       'Executive/ Senior Executive- Trade Marketing, Hygiene Products',
       'Executive/ Sr. Executive -IT',
       'Full Stack Developer (Python,React js)', 'HR Officer',
       'Head of Internal Control & Compliance (ICC) - SEVP/DMD',
       'Intern (Generative AI Engineering - 2D/3D Image Generation)',
       'Machine Learning (ML) Engineer',
       'Management Trainee - Mechanical',
       'Manager- Human Resource Management (HRM)\n', 'Marketing Officer',
       'Mechanical Designer', 'Mechanical Engineer',
       'Network Support Engineer', 'Project Coordinator (Civil)',
       'Senior Software Engineer', 'Senior iOS Engineer', 'Site Engineer',
       'Sr.Officer / Executive - Internal Audit',
       'System Administrator (Ope

TF-IDF vectorization

In [89]:
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=5000,
    ngram_range=(1, 2)
)
X_tfidf = tfidf.fit_transform(df_model['clean_text'])

In [90]:
X_tfidf.shape
len(y)

4740

Train & Split

In [91]:
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [92]:
#Model Naive Bayes , Logistic Regression
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
svm_model = LinearSVC()
svm_model.fit(X_train, y_train)

LinearSVC()

In [93]:
nb_pred = nb_model.predict(X_test)
lr_pred = lr_model.predict(X_test)
svm_pred = svm_model.predict(X_test)

Model Evaluation

In [94]:
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_pred))
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_pred))
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))

Naive Bayes Accuracy: 0.0
Logistic Regression Accuracy: 0.0
SVM Accuracy: 0.0


In [95]:
print(
    classification_report(
        y_test,
        svm_pred,
        target_names=label_encoder.classes_
    )
)

                                                                                         precision    recall  f1-score   support

                                                                            AI Engineer       0.00      0.00      0.00      34.0
                                                 Asst. Manager/ Manger (Administrative)       0.00      0.00      0.00      34.0
                                                         Business Development Executive       0.00      0.00      0.00      33.0
                                                                         Civil Engineer       0.00      0.00      0.00      34.0
                                                                          Data Engineer       0.00      0.00      0.00      33.0
                                                                  Data Science Engineer       0.00      0.00      0.00      34.0
                                                           Database Administrator (DBA)       0.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [96]:
confusion_matrix(y_test, svm_pred)

array([[0, 2, 0, 2, 1, 1, 1, 3, 0, 0, 0, 1, 2, 0, 0, 3, 1, 2, 0, 1, 4, 0,
        1, 1, 5, 1, 1, 1],
       [2, 0, 0, 0, 2, 0, 1, 1, 1, 0, 1, 1, 4, 1, 0, 0, 1, 3, 0, 2, 3, 0,
        2, 2, 5, 2, 0, 0],
       [1, 1, 0, 1, 1, 2, 0, 2, 1, 3, 1, 2, 3, 1, 0, 2, 1, 2, 0, 1, 0, 0,
        2, 3, 1, 1, 1, 0],
       [1, 2, 1, 0, 3, 0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 0, 2, 2, 1, 2, 2, 0,
        4, 1, 4, 1, 2, 1],
       [1, 1, 0, 1, 0, 1, 1, 0, 2, 3, 1, 1, 4, 0, 0, 0, 1, 2, 0, 2, 1, 0,
        0, 3, 2, 4, 1, 1],
       [2, 1, 0, 1, 3, 0, 0, 1, 1, 0, 2, 1, 2, 4, 0, 0, 1, 2, 0, 1, 1, 1,
        0, 2, 4, 2, 2, 0],
       [1, 0, 1, 1, 2, 0, 0, 2, 1, 1, 1, 0, 5, 3, 0, 2, 1, 1, 0, 0, 1, 1,
        0, 2, 2, 1, 3, 2],
       [1, 3, 0, 0, 2, 1, 2, 0, 2, 0, 0, 1, 1, 3, 0, 2, 1, 3, 3, 1, 2, 0,
        0, 1, 3, 1, 0, 1],
       [0, 2, 0, 1, 3, 1, 1, 2, 0, 1, 1, 1, 0, 1, 0, 0, 1, 3, 1, 1, 2, 0,
        1, 1, 2, 3, 3, 2],
       [2, 3, 0, 1, 2, 1, 0, 0, 1, 0, 1, 0, 3, 1, 0, 1, 2, 1, 2, 0, 0, 0,
        0, 3, 6, 

Prediciton

In [97]:
def predict_resume_category(resume_text):
    cleaned_text = clean_text(resume_text)
    vectorized_text = tfidf.transform([cleaned_text])
    predicted_label = svm_model.predict(vectorized_text)[0]
    return label_encoder.inverse_transform([predicted_label])[0]

In [98]:
sample_resume = """
Strong background in Python, machine learning, data analysis,
pandas, numpy, and scikit-learn. Experience in building ML models.
"""
predict_resume_category(sample_resume)

'Management Trainee - Mechanical'

Model

In [99]:
import joblib

In [100]:
joblib.dump(svm_model, 'svm_resume_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [101]:
svm_model = joblib.load('svm_resume_model.pkl')
tfidf = joblib.load('tfidf_vectorizer.pkl')
label_encoder = joblib.load('label_encoder.pkl')

Output

In [102]:
pip install streamlit

In [108]:
import streamlit as st
import joblib
import re
# Load saved objects
svm_model = joblib.load('svm_resume_model.pkl')
tfidf = joblib.load('tfidf_vectorizer.pkl')
label_encoder = joblib.load('label_encoder.pkl')
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()
st.title("Resume Classification System")
resume_input = st.text_area("Paste Resume Text Here")
if st.button("Predict Job Role"):
    if resume_input.strip() == "":
        st.warning("Please enter resume text")
    else:
        cleaned = clean_text(resume_input)
        vectorized = tfidf.transform([cleaned])
        prediction = svm_model.predict(vectorized)
        category = label_encoder.inverse_transform(prediction)[0]
        st.success(f"Predicted Job Role: {category}")

2026-01-02 06:40:22.775 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 06:40:22.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 06:40:22.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 06:40:22.780 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 06:40:22.782 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 06:40:22.783 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 06:40:22.784 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 06:40:22.785 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [114]:
def predict_real_and_best_role(resume_text):
    cleaned = clean_text(resume_text)
    vectorized = tfidf.transform([cleaned])

    # Get decision scores
    scores = svm_model.decision_function(vectorized)[0]

    # Softmax normalization
    scores = np.exp(scores) / np.sum(np.exp(scores))
    labels = label_encoder.classes_

    # Sort top 3
    top_idx = np.argsort(scores)[-3:][::-1]
    top_labels = labels[top_idx]
    top_scores = scores[top_idx]

    # Relative confidence
    relative = top_scores / top_scores[0]

    # Real output (truth)
    real_output = top_labels[0]

    # Recommendation logic
    if relative[1] >= 0.8:
        best_role = top_labels[1]
        reason = "Skills strongly overlap with this role as well"
    else:
        best_role = real_output
        reason = "Strongest match based on resume content"

    # Format output
    result = (
        f"🔹 Model Prediction (Real Output): {real_output}\n\n"
        f"✅ Best Role You Can Apply For: {best_role}\n"
        f"📌 Reason: {reason}\n\n"
        f"🔁 Other Suitable Roles:\n"
        f"- {top_labels[1]}\n"
        f"- {top_labels[2]}"
    )

    return result


In [118]:
import gradio as gr

gr.Interface(
    fn=predict_real_and_best_role,
    inputs=gr.Textbox(lines=10, label="Paste Your Resume Text Here"),
    outputs=gr.Textbox(lines=15, label="Role Analysis & Recommendation for your resume text"),
    title="Resume Classification & Job Recommendation System",
    description="Shows real model prediction and the best role you can apply for"
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1923489057adb3adb7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
